# Skillup AI Kaggle課題（第2週目）

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs as cd
import os
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier #クラス分類をする
from sklearn.metrics import log_loss, accuracy_score, precision_recall_fscore_support, confusion_matrix 
from sklearn.svm import SVC

In [2]:
os.chdir("C:\Anaconda3\envs\skillupai\Dataset")#読み込むデータのパスを指定
ds=pd.read_csv("Kaggle_SkillupAI.csv", engine="python")#CSVファイルの読み込み
display(ds.head())#上位10行の確認表示

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [3]:
#オリジナルデータを編集

#使用しない変数を削除
drop_col=['ID','name','category','currency','pledged','backers','usd pledged','usd_pledged_real','goal']
ds_use=ds.drop(drop_col, axis=1)

#開始日と締切日の間の日数を新たにperiodという変数として定義する
ds_use['deadline'] = pd.to_datetime(ds_use['deadline'], errors = 'coerce')
ds_use['launched'] = pd.to_datetime(ds_use['launched'], errors = 'coerce')
ds_use['period'] = (ds_use['deadline'] - ds_use['launched']).dt.days
ds_use = ds_use.drop(['deadline', 'launched'], axis=1)

#カテゴリ変数main_categoryとcountryをダミー変数に変換して結合する
pd.get_dummies(ds_use['main_category'])
ds_dummy_cate = pd.get_dummies(ds_use['main_category'])
pd.get_dummies(ds_use['country'])
ds_dummy_cou = pd.get_dummies(ds_use['country'])
ds_use = pd.concat([ds_use.drop(['main_category','country'],axis=1),ds_dummy_cate,ds_dummy_cou],axis=1)

#usd_goal_realを対数変換する
log_usd=np.log(ds_use['usd_goal_real'])
ds_use = pd.concat([ds_use.drop(['usd_goal_real'],axis=1),log_usd],axis=1)

#periodとusd_goal_realの欠損値を削除する
display(ds_use.isnull().sum())#欠損値の確認→欠損値は無いので、外れ値のみ削除する

#periodとusd_goal_realの外れ値（四分位範囲外の値）を削除する
col1=ds_use['period']
q1 = col1.describe()['25%']
q3 = col1.describe()['75%']
iqr=q3-q1
outlier_min1 = q1 - (iqr) * 1.5
outlier_max1 = q3 + (iqr) * 1.5
col1[col1 < outlier_min1] = None
col1[col1 > outlier_max1] = None

col2=ds_use['usd_goal_real']
q1 = col2.describe()['25%']
q3 = col2.describe()['75%']
iqr=q3-q1
outlier_min2 = q1 - (iqr) * 1.5
outlier_max2 = q3 + (iqr) * 1.5
col2[col2 < outlier_min2] = None
col2[col2 > outlier_max2] = None

ds_use['period']=col1
ds_use['usd_goal_real']=col2
ds_use = ds_use.dropna(how='any')

#periodとusd_goal_realの標準化（相関係数は約0.15と低いので、無相関化は実行しない）
ds_use["period"] = (ds_use["period"] - ds_use["period"].mean()) / ds_use["period"].std(ddof=0)
ds_use["usd_goal_real"] = (ds_use["usd_goal_real"] - ds_use["usd_goal_real"].mean()) / ds_use["usd_goal_real"].std(ddof=0)

#stateがsuccessfulとfailed以外のデータを削除
ds_suc=ds_use[ds_use['state']=='successful']
ds_not=ds_use[(ds_use['state']=='failed')]
ds_sum=pd.concat([ds_suc, ds_not])

state            0
period           0
Art              0
Comics           0
Crafts           0
Dance            0
Design           0
Fashion          0
Film & Video     0
Food             0
Games            0
Journalism       0
Music            0
Photography      0
Publishing       0
Technology       0
Theater          0
AT               0
AU               0
BE               0
CA               0
CH               0
DE               0
DK               0
ES               0
FR               0
GB               0
HK               0
IE               0
IT               0
JP               0
LU               0
MX               0
N,0"             0
NL               0
NO               0
NZ               0
SE               0
SG               0
US               0
usd_goal_real    0
dtype: int64

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#ロジスティック回帰の実行（）
y = ds_sum["state"].values
X = ds_sum.drop("state", axis=1).values
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X, y)

y_pred = clf.predict(X)
accuracy =  accuracy_score(y, y_pred)
print('正答率（Accuracy） = {:.3f}%'.format(100 * accuracy))

precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_pred)
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0]))
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0]))
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0]))

正答率（Accuracy） = 63.399%
適合率（Precision） = 64.802%
再現率（Recall） = 81.697%
F1値（F1-score） = 72.276%
